# Kapitel 3 - Erste Textklassifikation mit Naive Bayes 

## 3.1. Kapitelübersicht

In diesem Kapitel lernen wir unserer erstes Textklassifikationsverfahren kennen, das <b>Naive Bayes</b>-Verfahren. Es basiert auf dem Bayesschen Theorem, das von Thomas Bayes schon Mitte des 18. Jahrhunderts publiziert wurde. Das Naive Bayes Verfahren selbst ist seit den 1950er und 1960er ein intensiv untersuchtes Verfahren und wird selbst heute noch aufgrund seiner Simplizität als Textklassifizierungsverfahren genutzt.<br>
Es eignet sich hervorragend als "Einsteiger"-Verfahren, da man bei einem kleinen Datensatz die Klassifizierungsmethode selbst händisch und ohne Programmierwerkzeuge durchführen kann, was wir in 3.3. auch tun werden. Zuletzt werden wir in diesem Kapitel uns die Software-Bibliothek für Maschinelles Lernen in Python <b>Scikit-learn</b> anschauen und mithilfe von existierenden Modulen das Naive Bayes Verfahren implementieren und auf unseren Datensatz anwenden.

Am Ende dieses Kapitel werden wir folgende Themen behandelt und/oder vertieft haben:
- Naive Bayes, speziell das Multinomial Naive Bayes
- Bayes-Theorem/ Satz von Bayes
- Simples Feature Engineering mit dem Bag-of-Words Modell
- A-priori-Wahrscheinlichkeit
- TODO: scikit learn

## 3.2. Das Naive Bayes Verfahren

Das <b>Naive Bayes</b> Verfahren ist ein Klassifikationsverfahren, welches auf dem <b>Bayes-Theorem</b> (auch <b>"Satz von Bayes"</b> genannt) von Thomas Bayes basiert. Beim Naive Bayes Verfahren wird <u>naiv</u> (deshalb auch der Name) angenommen, dass alle Features der Eingabedaten (= input data) <b>unabhängig</b> voneinander sind. Obwohl das in der Realität selten der Fall ist, liefert Naive Bayes in der Praxis meist gute Werte.<br>
Das Naive Bayes Verfahren besteht nicht nur aus einem einzigen Algorithmus, sondern vielmehr aus einer Familie von Algorithmen, die alle auf dem gleichen Prinzip beruhen. Alle Algorithmen basieren jedoch auf der <b>probabilistischen Methode</b>. Wie wir im vorherigen Kapitel gesehen haben, bedeutet dies, dass Naive Bayes beispielsweise die Zugehörigkeit einer Kategorie zu einem Text für jede Kategorie berechnet und die höchste Wahrscheinlichkeit als Textkategorie zurückgibt. Die Wahrscheinlichkeiten werden durch das <b>Bayes-Theorem</b> ermittelt. Die Formel für das Theorem lautet: <br>
$ P(A|B) = \frac{P(B|A) \ \cdot \ P(A)}{P(B)} $ &nbsp; [<sup>1</sup>](#fn1) <br>



<hr style="border: 0.1px solid black;"/>
<span id="fn1" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">1</sup> &nbsp; Zwei anschauliche Rechenbeispiele mit Daten, die nicht aus Text bestehen, findet man auf der Wikipedia Seite vom <a href=https://de.wikipedia.org/wiki/Satz_von_Bayes>Satz von Bayes</a>.</span>

## 3.3. Ein einfaches Beispiel

Um uns mit dem Naive Bayes Verfahren vertraut zu machen, werden wir zunächst ein einfaches Beispiel[<sup>2</sup>](#fn2) durchrechnen, bevor wir das Verfahren auf unseren Datensatz anwenden. Dazu werden wir das <b>Multinomial Naive Bayes</b> Verfahren nutzen, eine der vielen Varianten von Naive Bayes. Dieses Verfahren wird genutzt, wenn eine multinomiale Verteilung der Daten vorliegt. Es funktioniert gut mit Textdaten, bei denen die Worthäufigkeiten berücksichtigt werden sollen. Bevor wir jedoch das Verfahren verwenden, schauen wir uns den Datensatz an. 

<hr style="border: 0.1px solid black;"/>
<span id="fn2" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">2</sup> &nbsp; Dieses Beispiel wurde von dem <a href=https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/>Naive Bayes Tutorial</a> der Plattform <b>MonkeyLearn</b> übernommen, übersetzt und leicht abgewandelt.</span>

In [12]:
import pandas as pd

news = {1 : ["Ein großartiges Spiel", "Sport"], 
        2 : ["Das Urteil ist beschlossen", "Kein Sport"],
        3 : ["Ein sehr faires Match", "Sport"],
        4 : ["Ein faires aber langweiliges Spiel", "Sport"],
        5 : ["Es ist ein knappes Urteil", "Kein Sport"],
        6 : ["Ein sehr knappes Spiel", "?"]}
simple_df = pd.DataFrame.from_dict(news, orient="index", columns=["Text", "Kategorie"])

simple_df

,Text,Kategorie
1,Ein großartiges Spiel,Sport
2,Das Urteil ist beschlossen,Kein Sport
3,Ein sehr faires Match,Sport
4,Ein faires aber langweiliges Spiel,Sport
5,Es ist ein knappes Urteil,Kein Sport
6,Ein sehr knappes Spiel,?


Wir sehen eine Tabelle, in der kurze Nachrichtenartikel in eine von zwei Kategorien eingeteilt wurden: "Sport" oder "Kein Sport". Der sechste Artikel "Ein sehr knappes Spiel" wurde in keine Kategorie eingeteilt. Unsere Aufgabe ist es jetzt, mithilfe des <b>Multinomial Naive Bayes</b> Klassifikationsverfahren die Wahrscheinlichkeiten zu berechnen, ob der Text "Ein sehr knappes Spiel" in die Kategorie "Sport" oder "Kein Sport" eingeteilt wird. Die <u>größere</u> Wahrscheinlichkeit bestimmt, zu welcher Kategorie der Satz gehört.
Mathematisch wird das folgenderweise ausgedrückt: <br>

$ P(Sport|Ein\ sehr\ knappes\ Spiel) $ <br>

$ P(Kein\ Sport|Ein\ sehr\ knappes\ Spiel) $

Wir haben nun ein Problem: Da unsere Daten aus Texten bestehen, können wir nicht mit ihnen rechnen und sie deshalb nicht in die Formel des Bayesschen Theorems einsetzen. Deshalb müssen wir zunächst <b>"Feature Engineering"</b> betreiben, einen zentralen Schritt bei der Nutzung von Machine Learning Algorithmen. 

<div class="alert alert-info">
<b>Exkurs:</b> Feature Engineering

TODO: verbessern!

<b>Feature Enigneering</b> beschreibt den Vorgang, die vorhandenen Daten in eine Form zu bringen, die ein maschinelles Lernverfahren nutzen kann. Grundsätzlich verwenden alle Algorithmen für maschinelles Lernen Eingabedaten, um Ausgaben zu erstellen. Diese Eingabedaten umfassen Features, die nicht immer direkt vom maschinellen Lernverfahren genutzt werden können und vorverarbeitet werden müssen. Dafür ist das Feature Engineering zuständig.<br>
Feature Engineering ist nicht immer der selbe Vorgang, sondern kann je nach Problemstellung anders aussehen. So ist das Problem bei Textdaten meistens, dass das maschinelle Lernverfahren mit der Repräsentation von Wörtern und Sätzen nichts anfangen kann, da es auf diese Form der Daten keine mathematische Rechenoperationen ausführen kann. Deshalb müssen die Textdaten in numerische Features transformiert werden (<b> Feature Transformation</b>). 

Im Grunde verfolgt Feature Engineering zwei Ziele: 
- Die Vorbereitung des richtigen Eingabedatensatzes, der mit den Anforderungen des maschinellen Lernalgorithmus kompatibel ist.
- Verbesserung der Leistung von Modellen für maschinelles Lernen.

</div>

Wir müssen uns dazu entschieden, welche Features wir aus den Texten extrahieren wollen. Würde unser Datensatz aus Personen bestehen und wir würden eine Klassifikation mit den Klassen "Gesund" und "Nicht Gesund" durchführen wollen, könnten wir als Features die Körpergröße, das Gewicht oder das Geschlecht nehmen und solche Features wie Vornamen, Lieblingsfarbe oder Sprachkenntnisse könnten wir weglassen.<br>
In unserem Fall haben wir jedoch keine numerischen Features, sondern nur Text. Um den Text in die Formel des <b>Bayesschen Theorems</b> einsetzen zu können, müssen wir den Text in Zahlen transformieren. Eine Möglichkeit bestünde darin, die einfachen Worthäufigkeiten jedes Textes zu nehmen. Dieses Verfahren nennt sich <b>Bag-of-Words</b>.

<div class="alert alert-info">
<b>Exkurs:</b> Bag-of-Words Modell

Ein <b>Bag-of-Words</b> Modell (kurz: BoW) ist eine Methode, um Features aus einem Text zu extrahieren, damit man diese für eine Modellierung, wie hier für einen Machine Learning-Algorithmus, verwenden kann. Die Idee hierbei ist, dass Dokumente, die eine ähnliche Anzahl der gleichen Wörter haben, ähnlichen Inhalt haben.

Der Bag-of-Words-Ansatz ist simpel und flexibel und kann vielseitig verwendet werden. Er beschreibt das Auftreten von Wörtern in einem Dokument und besteht aus einem Vokabular der bekannten Wörter und einem Maß für das Vorkommen der bekannten Wörter. Die Bezeichnung "bag" (deutsch: <i>Sack</i>) soll darauf hinweisen, dass alle Informationen über die Struktur oder Reihenfolge der Wörter im Dokument verworfen werden. Es ist also nicht mehr möglich, die Lage eines Wortes nach der Umwandlung eines Textes in einen bag-of-words zu bestimmen.

Das Bag-of-Words-Modell kann erweitert werden, etwa durch <b>tf-idf</b> oder <b>N-Gramme</b> (n-grams).
</div>

Bevor wir aber unseren Text in ein <b>Bag-of-words</b>-Modell überführen, setzen wir zunächst den Satz "Ein sehr knappes Spiel" in die Formel für das <b>Bayes-Theorem</b> ein. Wir erhalten somit:

$ P(Sport|Ein\ sehr\ knappes\ Spiel) = \frac{P(Ein\ sehr\ knappes\ Spiel|Sport) \ \cdot \ P(Sport)}{P(Ein\ sehr\ knappes\ Spiel)}  $ <br>


$ P(Kein\ Sport|Ein\ sehr\ knappes\ Spiel) = \frac{P(Ein\ sehr\ knappes\ Spiel|Kein\ Sport) \ \cdot \ P(Kein\ Sport)}{P(Ein\ sehr\ knappes\ Spiel)}  $ <br>

Da wir bei unserem Vergleich nur herausfinden wollen, welche Kategorie die höhere Wahrscheinlichkeit hat, können wir zur Vereinfachung den Divisor weglassen und erhalten somit die folgenden gekürzten Formeln:

$ P(Sport|Ein\ sehr\ knappes\ Spiel) = P(Ein\ sehr\ knappes\ Spiel|Sport) \ \cdot \ P(Sport)  $ <br>


$ P(Kein\ Sport|Ein\ sehr\ knappes\ Spiel) = P(Ein\ sehr\ knappes\ Spiel|Kein\ Sport) \ \cdot \ P(Kein\ Sport)  $ <br>


Jetzt müssten wir eigentlich nur noch zählen, wie oft der Satz "Ein sehr knappes Spiel" in der Kategorie "Sport" oder "Kein Sport" auftritt und das durch die Gesamtanzahl aller Texte teilen. Somit würden wir $ P(Ein\ sehr\ knappes\ Spiel|Sport) $ oder $ P(Ein\ sehr\ knappes\ Spiel|Kein\ Sport) $ berechnen können. Das Problem ist jedoch, dass dieser Satz in den anderen Texten gar nicht vorkommt und die Wahrscheinlichkeit für beide Kategorien $ P = 0 $ wäre. Unser Modell wäre nur sinnvoll, wenn wir sehr große Texte hätten und die Texte den exakten Satz beinhalten würden.<br>

Wir müssen uns deshalb den <i>naiven</i> Teil vom <b>Naive Bayes</b> Verfahren zu Nutze machen. Anstatt dass wir den gesamten Satz betrachten, betrachten wir die einzelnen Wörter <u>unabhängig</u> voneinander. Sätze wie "ich gehe nicht nach hause" und "nicht nach hause gehe ich" werden nicht unterschieden, für unser Modell sind es die gleichen Sätze, da sie die gleichen Wörter beinhalten.<br>

Mithilfe dieser Annahme können wir nun die Wörter in $ P(Ein\ sehr\ knappes\ Spiel) $ einsetzen und erhalten:<br>

$ P(Ein\ sehr\ knappes\ Spiel) = P(Ein) \cdot P(sehr) \cdot P(knappes) \cdot P(Spiel) $ <br>

Das setzen wir nun in $ P(Ein\ sehr\ knappes\ Spiel|Sport) $ ein und erhalten somit:<br>

$ P(Ein\ sehr\ knappes\ Spiel|Sport) = P(Ein|Sport) \cdot P(sehr|Sport) \cdot P(knappes|Sport) \cdot P(Spiel|Sport) $ <br>



Jetzt können wir endlich unsere Wahrscheinlichkeiten berechnen, indem wir die Wörter in unseren Daten zusammenzählen. Dafür berechnen wir zunächst für jede der beiden Kategorien ihre <i>A-priori</i>-Wahrscheinlichkeit: Die Wahrscheinlickeit, dass ein Text in unserem Datensatz eine der beiden Kategorien angehört,[<sup>3</sup>](#fn3) ist für $ P(Sport) $ somit $ \frac{3}{5} $ und für $ P(Kein\ Sport) $ ist sie $ \frac{2}{5}$.


<hr style="border: 0.1px solid black;"/>
<span id="fn3" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">3</sup> &nbsp; Ausgenommen wird hier natürlich der Satz "Ein sehr knappes Spiel", der (noch) keiner Kategorie zugewiesen ist und deshalb nicht berücksichtigt wird.</span>

<div class="alert alert-info">
<b>Exkurs:</b> <i>A-priori</i>-Wahrscheinlichkeit<br>

Die A-priori-Wahrscheinlichkeit ist ein <b>Wahrscheinlichkeitswert</b>, der aufgrund von Vorwissen gewonnen werden kann. Möchte man beispielsweise die Wahrscheinlichkeit eines Würfelwurfs berechnen, kann man schon vor dem Wurf sagen, dass die Wahrscheinlichkeit des Auftretens einer bestimmten Zahl $ \frac{1}{6} $ ist, da man vorher wusste, dass der Würfel sechs Seiten hat. Das gleiche gilt für einen Münzwurf, da wir dort wissen, dass eine Münze nur zwei Seiten hat und deshalb die Wahrscheinlichkeit $ \frac{1}{2} $ ist (natürlich nur wenn man davon ausgeht, dass eine Münze bei dieser Annahme nicht auf der Kante landen darf).

In [24]:
simple_df

,Text,Kategorie
1,Ein großartiges Spiel,Sport
2,Das Urteil ist beschlossen,Kein Sport
3,Ein sehr faires Match,Sport
4,Ein faires aber langweiliges Spiel,Sport
5,Es ist ein knappes Urteil,Kein Sport
6,Ein sehr knappes Spiel,?


Nun berechnen wir nacheinander die Wahrscheinlichkeit $ P $ der einzelnen Wörter, angefangen mit $ P(Ein|Sport) $. Dafür zählen wir zunächst, wie oft das Wort "Ein" in Texten der Kategorie "Sport" vorkommt (3 Mal). Das teilen wir durch die Gesamtanzahl der Wörter aus den Texten der Kategorie "Sport" (12 Wörter). Wir erhalten also $ P(Ein|Sport) = \frac{3}{12}$.<br>
Diese Berechnung führen wir nun für jedes der Wörter des Satzes "Ein sehr knappes Spiel" durch und erhalten folgende Ergebnisse:<br>

$ P(sehr|Sport) = \frac{1}{12}$<br>
$ P(knappes|Sport) = \frac{0}{12} = 0 $<br>
$ P(Spiel|Sport) = \frac{2}{12}$<br>

Wir haben nun jedoch ein Problem: Das Wort "knappes" kommt in keinem der Texte aus der Kategorie "Sport" vor und die Wahrscheinlichkeit ist somit 0. Würden nun aber die Berechnung $ P(Ein|Sport) \cdot P(sehr|Sport) \cdot P(knappes|Sport) \cdot P(Spiel|Sport) $ ausführen, würde der gesamte Ausdruck 0 werden, da $ P(Ein|Sport) \cdot P(sehr|Sport) \cdot 0 \cdot P(Spiel|Sport) = 0$.<br>
Wir müssen dieses Problem umgehen, da wir ansonsten nur Sätze klassifizieren könnten, bei denen alle Wörter in jeder Kategorie vorkommen. Wir wenden das sogenannte <b>Laplace smoothing</b> an, was bedeutet, dass wir zu jedem gezählten Wort 1 addieren, damit es nie 0 werden kann. Damit wäre es jedoch möglich, dass wir eine Wahrscheinlichkeit > 1 erhielten, wenn alle Texte einer Kategorie nur aus einem Wort bestünden. Würden alle Texte der Kategorie "Sport" also aus dem einzelnen Wort "Spiel" bestehen, wäre $ P(Spiel|Sport) = \frac{3+1}{3} = \frac{4}{3} > 1$. Dieser Fall ist sehr unwahrscheinlich, trotzdem addieren wir im Divisor die Anzahl aller möglichen Wörter, nicht nur die der Kategorie "Sport", sondern auch der Kategorie "Kein Sport". Dieses Vokabular besteht in unserem Fall aus 14 Wörtern.<br>
$ P(Ein|Sport) $ mit Laplace Smoothing wäre also $ P(Ein|Sport)  = \frac{3+1}{12+14} = \frac{4}{26} $.

In [27]:
smaller_simple_df = simple_df[:-1]

results = set()
smaller_simple_df['Text'].str.lower().str.split().apply(results.update)
print("Vokabular: " + str(results) +"\n")
print("Größe des Vokabulars: " + str(len(results)))

Vokabular: {'sehr', 'faires', 'match', 'knappes', 'langweiliges', 'urteil', 'aber', 'spiel', 'beschlossen', 'ist', 'großartiges', 'es', 'das', 'ein'}

Größe des Vokabulars: 14


Nun berechnen wir für jedes Wort des Satzes "Ein sehr knappes Spiel" die Wahrscheinlichkeit für beide Kategorien.

<table style="width:75%" align="left">
  <tr>
    <th>Wort</th>
    <th>$ P(wort|Sport) $</th>
    <th>$ P(wort|Kein\ Sport) $</th>
  </tr>
  <tr>
    <td>Ein</td>
    <td>$ \frac{3+1}{12+14} = \frac{4}{26}$</td>
    <td>$ \frac{1+1}{9+14} = \frac{2}{23}$</td>
  </tr>
  <tr>
    <td>sehr</td>
    <td>$ \frac{1+1}{12+14} = \frac{2}{26}$</td>
    <td>$ \frac{0+1}{9+14} = \frac{1}{23}$</td>
  </tr>
    <tr>
    <td>knappes</td>
    <td>$ \frac{0+1}{12+14} = \frac{1}{26}$</td>
    <td>$ \frac{1+1}{9+14} = \frac{2}{23}$</td>
  </tr>
    <tr>
    <td>Spiel</td>
    <td>$ \frac{2+1}{12+14} = \frac{3}{26} $</td>
    <td>$ \frac{0+1}{9+14} = \frac{1}{23}$</td>
  </tr>
</table><br>

Zuletzt multiplizieren wir die Wahrscheinlichkeiten für jede Kategorie:<br>

$  P(Ein|Sport) \cdot P(sehr|Sport) \cdot P(knappes|Sport) \cdot P(Spiel|Sport) \cdot P(Sport) 
\\ = \frac{4}{26} \cdot \frac{2}{26} \cdot \frac{1}{26} \cdot \frac{3}{26} \cdot \frac{3}{5} 
\\ = 0,000031512$ <br>

$  P(Ein|Kein\ Sport) \cdot P(sehr|Kein\ Sport) \cdot P(knappes|Kein\ Sport) \cdot P(Spiel|Kein\ Sport) \cdot P(Kein\ Sport) 
\\ = \frac{2}{23} \cdot \frac{1}{23} \cdot \frac{2}{23} \cdot \frac{1}{23} \cdot \frac{2}{5} 
\\ = 0,000005718$ <br>

Da 0,000031512 > 0,000005718, ist die Wahrscheinlichkeit, dass der Satz "Ein sehr knappes Spiel" der Kategorie "Sport" angehört, höher.

## 3.4. Naive Bayes mit Scikit learn

Im vorherigen Beispiel hatten wir einen Datensatz, der aus nur fünf sehr kurzen Texten bestand und nur zwei Kategorien hatte. Damit war es sehr einfach, das Naive Bayes Klassifikationsverfahren händisch zu berechnen. Die meisten Datensätze sind jedoch um einiges größer. Dort ist es von Vorteil, das Naive Bayes Verfahren in Python (oder einer anderen Programmiersprache) zu implementieren oder auf die Implementierung von der Software-Bibliothek <b>Scikit learn</b> zurückzugreifen.

TODO: kurzer scikit learn guide

In [23]:
import pandas as pd

news = {1 : ["Ein großartiges Spiel", "Sport"], 
        2 : ["Das Urteil ist beschlossen", "Kein Sport"],
        3 : ["Ein sehr faires Match", "Sport"],
        4 : ["Ein faires aber langweiliges Spiel", "Sport"],
        5 : ["Es ist ein knappes Urteil", "Kein Sport"],
        6 : ["Ein sehr knappes Spiel", "Kein Sport"]}
corpus = pd.DataFrame.from_dict(news, orient="index", columns=["Text", "Kategorie"])

corpus = corpus[:-1]

In [24]:
corpus

,Text,Kategorie
1,Ein großartiges Spiel,Sport
2,Das Urteil ist beschlossen,Kein Sport
3,Ein sehr faires Match,Sport
4,Ein faires aber langweiliges Spiel,Sport
5,Es ist ein knappes Urteil,Kein Sport


In [59]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np


#ICH: hier weiter, count vectorizer einzeln und prediction problem lösen

vec = CountVectorizer().fit_transform(corpus["Text"]).toarray()
Y = LabelEncoder().fit_transform(corpus["Kategorie"])

"""X_train, X_test, y_train, y_test = train_test_split(vec, 
                                                    Y,
                                                    test_size=0.1, 
                                                    random_state=42, 
                                                    shuffle=True)"""


#X_train, X_test, y_train, y_test = train_test_split(vec, Y, random_state = 0)


#count_vect = CountVectorizer()
#X_train_counts = count_vect.fit_transform(X_train)
#tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB(alpha=0.01).fit(vec, Y)
#classifier = MultinomialNB()


#X_train = simple_df["Text"]
#y_train = simple_df["Kategorie"]
#X_test = "Ein sehr knappes Spiel"

#mnb_clf = MultinomialNB(alpha=0.01)
#mnb_clf.fit(X_train, y_train)
#mnb_pred = mnb_clf.predict(X_test)

SyntaxError: invalid syntax (<ipython-input-59-7186de691bdc>, line 8)

In [47]:
print(vec)
print(Y)

[[0 0 0 1 0 0 1 0 0 0 0 0 1 0]
 [0 1 1 0 0 0 0 1 0 0 0 0 0 1]
 [0 0 0 1 0 1 0 0 0 0 1 1 0 0]
 [1 0 0 1 0 1 0 0 0 1 0 0 1 0]
 [0 0 0 1 1 0 0 1 1 0 0 0 0 1]]
[1 0 1 1 0]


In [48]:
a = CountVectorizer().fit_transform(["Ein sehr knappes Spiel"])
a

<1x4 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [58]:
print(clf.predict(vec(["Ein sehr knappes Spiel"])))

TypeError: 'numpy.ndarray' object is not callable

In [33]:
Y

array([1, 0, 1, 1, 0])

https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f


https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

https://stackabuse.com/text-classification-with-python-and-scikit-learn/

liegt an unseren Daten (die den Daten unseres Korpus(?) ähneln). Unsere Daten sind nämlich <b>diskrete</b> Daten. 




Diskrete vs continous... 

Bag of words hat einige Probleme:
- goodnote
- https://machinelearningmastery.com/gentle-introduction-bag-words-model/

Probleme/Erläuterungen für Bericht:
- wie weit Feature Engineering beschreiben, wenn Fokus später auf DL liegt?
- Wieviele Möglichkeiten von BoW beschreiben?
- Beispiel von Monkey Learning war gut, musste aber übersetzt werden. hier ist die deutsche sprache wieder problematisch, wegen den konjugation etc. einige wörter wurden ausgetauscht
- die idee bei diesem teil war es, dass sehr einfach ein paar grundzüge und herangehensweisen von klassifikation erklärt werden können
    - wahl des verfahrens: wurde hier abgenommen, wird später dann genauer erläutert
    - inhalt: was ist das verfahren überhaupt? eine auseinandersetzung mit verfahren ist vorteilhaft, es wurde aber wert darauf gelegt, dass es nicht zu tief in die materie geht, vor allem was den mathematischen aspekt angeht. deshalb wurde die mathematik einfach gehalten und ausreichend erklärt.
    - mathematik: jedes verfahren basiert natürlich auf einer mathematik. geisteswissenschaftler, also auch dhler, scheuen sich aber etwas davor, da der umgang mit formeln nicht geübt ist wie bei informatikern, mathematiker oder allgemein naturwissenschaftlern. hier ist es also wichtig, die mathematik gut und einfach zu erklären und vor allem mathematik, die nicht gebraucht wird, wegzulassen. zu viel sollte m.E. jedoch nicht weggelassen werden, da ein basis verständnis hilft, die verfahren zu verstehen. da in späteren kapiteln der anteil der mathematik zunehmen würde, wurden hier basis rechnung ausführlich erklärt, damit der leser ein verständnis dafür bekommt, ohne abgeschreckt zu werden.
    - info boxen: für diese wurde sich bewusst entschieden, da sie nicht direkt in den text gepasst haben, aber wichtig sind. feature engineering und bow wurden mit einem satz im "fließtext" erklärt und in der info box etwas ausführlicher beleuchtet.
    - einfaches bespiel ohne programmierung: ich finde, dass rechenbeispiele sehr gut helfen, ein verfahren zu verstehen. hätte ich hier direkt mit einem sci kit learn beispiel angefangen, wäre ein tieferes verständnis von naive bayes nicht gegeben. der leser könnte zwar daten klassifizieren, was sci kit learn aber eigentlich im hintergrund macht, bleibt ihm verwehrt. deswegen wurde hier sehr sehr ausfürlich das verfahren per Hand gerechnet. der leser versteht so, was eigentlich gemacht werden und kann sich vorstellen, wieso ein programmiermehtode sinnvoll ist, vor allem bei vielen Daten. dieses einfache beispiel ist sinnbildlich für die funktionsweise von textklassifikationsverfahren. es war jedoch klar/ es wurde sich bewusst dazu entschieden, nicht für jedes weitere verfahren ein händisches rechenbeispiel zu liefern, da dies zu viel des guten wäre und sich zu vieles wiederholen würde. in nächsten verfahren, logistic regression, soll dies noch einmal wiederholt, damit sollte eine gute intuition gegeben sein, was der computer da eigentlich rechnet.
    - sci kit learn:
    - datensatz mit naive bayes:
    - erweiterung mit tf idf und n-gramme: ICH: vllt erst nächstes kapitel?

Quellen:
- https://machinelearningmastery.com/gentle-introduction-bag-words-model/
- https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/